In [ ]:
!pip install datasets
from IPython.display import clear_output
clear_output()

In [ ]:
import jax
import jax.numpy as np
from numpy import testing
import math

def test(func, func_and_jac, *params):
  testing.assert_allclose(func(*params), func_and_jac(*params)[0])
  for i in range(len(params)):
    print(f"Testing Jacobian of parameter {i}")
    testing.assert_allclose(jax.jacfwd(func, argnums=i)(*params), func_and_jac(*params)[i + 1])
  print("All passed!")

In [ ]:
# Reference implementation of a func_and_jac function.
def add_and_jac(x, y):
  output = x + y
  x_jac = np.eye(math.prod(x.shape)).reshape(*x.shape, *x.shape)
  x_jac = np.broadcast_to(x_jac, (*output.shape, *x.shape))
  y_jac = np.eye(math.prod(y.shape)).reshape(*y.shape, *y.shape)
  y_jac = np.broadcast_to(y_jac, (*output.shape, *y.shape))
  return output, x_jac, y_jac

x, y = np.arange(24).reshape(2, 3, 4).astype(float)
test(np.add, add_and_jac, x, y)

In [ ]:
def matmul_and_jac(x, y):
  # TODO: define the Jacobian for the matmul function.
  ### BEGIN IMPLEMENTATION ###
    output = np.matmul(x, y)

    m, n = x.shape
    n2, p = y.shape
    x_jac = np.zeros((m, p, m, n))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                x_jac = x_jac.at[i, j, i, k].set(y[k, j])

    y_jac = np.zeros((m, p, n, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                y_jac = y_jac.at[i, j, k, j].set(x[i, k])

    return output, x_jac, y_jac

  #pass

x = np.arange(12).reshape(3, 4).astype(float)
y = np.arange(20).reshape(4, 5).astype(float)
test(np.matmul, matmul_and_jac, x, y)

In [ ]:
def affine(x, w, b):
  return np.dot(x, w) + b

def affine_and_jac(x, w, b):
  # TODO: use add_and_jac and matmul_and_jac
  # to define the Jacobian of the affine function
  ### BEGIN IMPLEMENTATION ###

  output, xjx, wjw = matmul_and_jac(x, w)

  ret, dxw, bias = add_and_jac(output, b)

  return ret, xjx, wjw, bias

  #pass

x = np.arange(12).reshape(3, 4).astype(float)
w = np.arange(20).reshape(4, 5).astype(float)
b = np.arange(5).reshape(1, 5).astype(float)
test(affine, affine_and_jac, x, w, b)

In [ ]:
def relu(x):
  return np.maximum(x, 0)

def relu_and_jac(x):
  # Some libraries define $d relu(0) / dx$ to be 0.5.
  x_jac = (np.diagflat(x > 0) + np.diagflat(x == 0) / 2).reshape(*x.shape, *x.shape)
  return np.maximum(x, 0), x_jac

def logsoftmax(x):
  stabilized_x = x - np.max(x, axis=1, keepdims=True)
  return stabilized_x - np.log(np.sum(np.exp(stabilized_x), axis=1, keepdims=True))

def logsoftmax_and_jac(x):
  # TODO: define the Jacobian of the log_softmax function
  ### BEGIN IMPLEMENTATION ###

    stabilized_x = x - np.max(x, axis=1, keepdims=True)
    softmax = np.exp(stabilized_x) / np.sum(np.exp(stabilized_x), axis=1, keepdims=True)
    logs = stabilized_x - np.log(np.sum(np.exp(stabilized_x), axis=1, keepdims=True))

    m, n = x.shape
    jacobian = np.ones((m, n, m, n))

    for i in range(m):
        for j in range(n):
            for k in range(m):
                for l in range(n):
                    if i == k:
                        if j == l:
                            jacobian = jacobian.at[i, j, k, l].set(1 - softmax[i, j])
                        else:
                            jacobian = jacobian.at[i, j, k, l].set(-softmax[i, l])
                    else:
                        jacobian = jacobian.at[i, j, k, l].set(0)

    return logs, jacobian

x = np.arange(12).reshape(3, 4).astype(float)
test(logsoftmax, logsoftmax_and_jac, x)

In [ ]:
# Here are some utilities to deal with nested parameters and composing the Jacobians we defined.
def tree_map(func, *trees: np.ndarray | tuple[np.ndarray], **kwargs):
  if not isinstance(trees[0], tuple):
    return func(*trees, **kwargs)
  else:
    return tuple(tree_map(func, *subtrees, **kwargs) for subtrees in zip(*trees))

def sequential(func_and_jac_seq):
  def func_and_jac(x, params_seq):
    params_jac = ()
    x, x_jac, _ = add_and_jac(x, np.zeros_like(x))
    for func_and_jac, params in zip(func_and_jac_seq, params_seq):
      x, x_layer_jac, *param_layer_jacs = func_and_jac(x, *params)
      params_jac = tree_map(lambda param: np.einsum("...kl,klmn->...mn", x_layer_jac, param), params_jac)
      x_jac = np.einsum("...kl,klmn->...mn", x_layer_jac, x_jac)
      params_jac += (tuple(param_layer_jacs),)
    return x, x_jac, params_jac
  return func_and_jac

# Here is a 2-layer MLP
def mlp(x, params):
  affine1_params, _, affine2_params, _ = params
  return logsoftmax(affine(relu(affine(x, *affine1_params)), *affine2_params))

# ...and its Jacobian!
mlp_and_jac = sequential(
    [affine_and_jac, relu_and_jac, affine_and_jac, logsoftmax_and_jac]
)

import numpy

mlp_params = (
    (
      numpy.random.randn(10,100), # first matmul
      numpy.random.randn(1, 100) # first bias
    ),
    (), # relu has no params
    (
      numpy.random.randn(100,10), # final matmul
      numpy.random.randn(1, 10) # final bias
    ),
    (), # logsoftmax has no params
)

x = numpy.random.randn(3,10)
y = np.eye(10)[numpy.random.randint(0, 10, size=3)]
print("Testing MLP Jacobian")
tree_map(
  numpy.testing.assert_allclose,
  jax.jacfwd(mlp, argnums=1)(x, mlp_params),
  mlp_and_jac(x, mlp_params)[2],
  atol=1e-5
)
print("Passed!")

In [ ]:
def cross_entropy_loss(params, x, y):
  logprobs = mlp(x, params)
  return -np.mean(np.sum(logprobs * y, axis=1))

def cross_entropy_loss_and_grad(logprobs, y):
    return -np.mean(np.sum(logprobs * y, axis=1)), -y / logprobs.shape[0]

x = numpy.random.randn(3,10)
y = np.eye(10)[numpy.random.randint(0, 10, size=3)]
logprobs, x_jac, params_jac = mlp_and_jac(x, mlp_params)
loss, output_grad = cross_entropy_loss_and_grad(logprobs, y)

 # TODO: Write the function to compute the parameter gradients
    # using the output_grad and each parameter jacobian.
    # BEGIN IMPLEMENTATION

param_grads = tree_map(lambda jac: np.einsum('bi,bijk->jk', output_grad, jac), params_jac)

print("Testing MLP grad")
jax_grads = jax.grad(cross_entropy_loss)(mlp_params, x, y)
tree_map(numpy.testing.assert_allclose, param_grads, jax_grads, atol=1e-5)
print("All tests passed!")

In [ ]:
# Nothing to do in this cell, just a reference implementation.

import jax.numpy as jnp
from jax import random, grad, jit
import matplotlib.pyplot as plt

# 1. Define the MLP Model
def init_mlp_params(layer_sizes, key):
    """Initialize the parameters for an MLP."""
    params = []
    keys = random.split(key, len(layer_sizes) - 1)
    for in_size, out_size, key in zip(layer_sizes[:-1], layer_sizes[1:], keys):
        weight_key, bias_key = random.split(key)
        W = random.normal(weight_key, (in_size, out_size)) * jnp.sqrt(2.0 / in_size)
        b = jnp.zeros(out_size)
        params.append((W, b))
    return params

def mlp(params, x):
    """Forward pass for an MLP."""
    activations = x
    for W, b in params[:-1]:
        activations = jax.nn.relu(jnp.dot(activations, W) + b)
    final_W, final_b = params[-1]
    return jnp.dot(activations, final_W) + final_b

# 2. Define the loss function
def loss_fn(params, x, y):
    predictions = mlp(params, x)
    return jnp.mean((predictions - y) ** 2)

# 3. Generate data (sine function between -π and π)
key = random.PRNGKey(0)
x_train = jnp.linspace(-jnp.pi, jnp.pi, 256).reshape(-1, 1)
y_train = jnp.sin(x_train)

# 4. Initialize the model parameters
layer_sizes = [1, 64, 64, 1]  # MLP with two hidden layers
params = init_mlp_params(layer_sizes, key)

# 5. Gradient Descent update rule
learning_rate = 0.01  # Set a suitable learning rate

@jit
def update(params, x, y):
    """Performs one update step using plain gradient descent."""
    grads = grad(loss_fn)(params, x, y)  # Compute gradients
    new_params = [(W - learning_rate * dW, b - learning_rate * db)
                  for (W, b), (dW, db) in zip(params, grads)]
    return new_params

# 6. Train the model
num_epochs = 10000
for epoch in range(num_epochs):
    params = update(params, x_train, y_train)
    if epoch % 1000 == 0:
        loss = loss_fn(params, x_train, y_train)
        print(f'Epoch {epoch}, Loss: {loss}')

# 7. Plot the results
x_test = jnp.linspace(-jnp.pi, jnp.pi, 256).reshape(-1, 1)
y_pred = mlp(params, x_test)

plt.plot(x_train, y_train, label='True function (sin)')
plt.plot(x_test, y_pred, label='MLP approximation', linestyle='dashed')
plt.legend()
plt.show()

In [ ]:
# Download data - do not change
import requests
import numpy as np
from tqdm.notebook import tqdm
import time

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
response.raise_for_status() # Raise an exception for invalid HTTP status codes
text_data = response.text
len(text_data), text_data[:100]


In [ ]:

# preprocessing - do not change
text_data = response.text
text_data = text_data.replace(',',' , ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
text_data = text_data.replace('  ', ' ')
text_data = text_data.replace('\n\n','\n').replace('\n',' </s> <s> ')
text_data = '<s> ' + text_data + ' </s>'
len(text_data), text_data[:100]

In [ ]:
data = text_data.split(' ')
vocab = list(set(data))
revmap = {v:k for k,v in enumerate(vocab)} # dictionary to map words to indices


In [ ]:
counts = np.zeros(shape=(len(vocab), len(vocab)))
assert counts.shape == (18118, 18118)

span_length = 5
start_time = time.time()

# IMPLEMENT HERE

for i,w in tqdm(enumerate(data), total=len(data)):
  # Hint: calculate co-occurrences in each window

  ind = revmap[w]
  begin = i - span_length
  fin = i + span_length + 1

  if begin < 0:
      begin = 0
  if fin > len(data):
      fin = len(data)

  for j in range(begin, fin):
      if i != j:
          context_w = revmap[data[j]]
          counts[ind, context_w] += 1

# Hint: calculate PPMI values using co-occurrences
cocurr = counts / np.sum(counts)
pi = np.sum(counts, axis=1) / np.sum(counts)
pj = np.sum(counts, axis=0) / np.sum(counts)

cocurr = np.clip(cocurr, 1e-10, None)
pi = np.clip(pi, 1e-10, None)
pj = np.clip(pj, 1e-10, None)

ppmi = np.maximum(0, np.log(cocurr / (pi[:, np.newaxis] * pj[np.newaxis, :]))) #make shapes compatible

# Hint: Find L2 norms to convinently calculate cosine similiarity scores
ppmi_norms = np.linalg.norm(ppmi, axis=1)

### END OF IMPLEMENTATION ####
print("--- %s seconds ---" % (time.time() - start_time))
# Note down this Training Time

assert ppmi.shape == (len(vocab), len(vocab))
assert counts[revmap['First'],revmap['Citizen:']] == 44.
np.testing.assert_allclose(ppmi[revmap['First'],revmap['Citizen:']], 4.024266, rtol=1e-3)

In [ ]:
def get_nearest_neighbours(word_or_vector, vectors=ppmi, top_k=5, vocab=vocab,
                           norms=ppmi_norms):
  # IMPLEMENT HERE
    if type(word_or_vector) == str:
      entry = revmap[word_or_vector]
      vectarget = vectors[entry]
    else:
      vectarget = word_or_vector

    target_norm = np.linalg.norm(vectarget)
    cos = np.dot(vectors, vectarget) / (norms * target_norm)

    sorted_word = np.argsort(-cos)

    nearest_neighbours = []
    for i in range(top_k):
        wordval = sorted_word[i]
        nearest_neighbours.append(vocab[wordval])

    return nearest_neighbours
  # Hint: L2 norms can be used for ease of calculating cosine similarity.
  # Hint: avoid iterating over vocabulary and use np.dot matrix multiplication.
  # Hint: sort by cosine similarity scores.

In [ ]:
get_nearest_neighbours('he')
print(get_nearest_neighbours('he'))
assert 'he' in get_nearest_neighbours('he')

In [ ]:
# Download and preprocessing data - do not change
import requests
import numpy as np
from tqdm.notebook import tqdm

url = "http://download.tensorflow.org/data/questions-words.txt"
response = requests.get(url)
response.raise_for_status() # Raise an exception for invalid HTTP status codes
analogy = response.text.split('\n')

# filtering out analogies that do not exist in our vocabulary
task = []
for line in tqdm(analogy, total=len(analogy)):
  row = line.split(' ')
  if len(row) != 4 or line.startswith(':'):
    continue
  if sum(_ in vocab for _ in row) == 4:
    task.append(row)
len(task), task[-1]

In [ ]:
# Inference over Analogy Benchmark - do not change
start_time = time.time()
acc = 0.
for a,b,c,d in tqdm(task[:50]): # Note down this Inference Time
  d_ = get_nearest_neighbours(
        word_or_vector=ppmi[revmap[a]]+ppmi[revmap[b]]-ppmi[revmap[c]],
        top_k=100
      )
  if d in d_:
    acc += 1
print('inference time:', time.time() - start_time)
acc/len(task)

In [ ]:
import scipy.special
import numpy as np

D = 50 # embedding_dimensions
w_vectors = np.random.rand(len(vocab), D) # target
c_vectors = np.random.rand(len(vocab), D) # context and noise

In [ ]:
start_time = time.time()
LR = 1e-2 # learning_rate
K = 15 # num_negatives
span_length = 5 # number of words before and after target in context
num_iter = 500_000 # number of iterations or update steps

losses = []
for iter in tqdm(range(num_iter)): # # Note down this Training Time
  # IMPLEMENT HERE
  # Hint: Pick a random target word from data, then iterate over each positive word in its context (using span_length)
  # Hint: Fetch target, context, and negatives from w_vectors and c_vectors.
  # Hint: Calculate gradient updates as derived in Q2.
  # Hint: You may use scipy.special.expit function to implement sigmoid.
  # Hint: Apply gradient updates to parts of w_vectors and c_vectors properly

    target = np.random.randint(0, len(data))
    target_word = data[target]

    index = revmap[target_word]

    start = max(0, target - span_length)
    end = min(len(data), target + span_length + 1)

    contexts = []
    for j in range(start, end):
        if j != target:
            contexts.append(data[j]) #grab contexts words around it

    contexts_word = []
    for word in contexts:
        contexts_word.append(revmap[word])

    neg_words = np.random.randint(0, len(vocab), size=K) #get neg samples

    targetv = w_vectors[index] #get vecs
    context_v = c_vectors[contexts_word]
    negative_v = c_vectors[neg_words]

    positive_scores = np.dot(context_v, targetv)  #get pos score and run sigmoid
    sigmoid = scipy.special.expit(positive_scores)

    negative_scores = np.dot(negative_v, targetv)   #get neg and run sig
    neg_sigmoid = scipy.special.expit(-negative_scores)

    sigmoid = np.clip(sigmoid, 1e-7, 1 - 1e-7)
    neg_sigmoid = np.clip(neg_sigmoid, 1e-7, 1 - 1e-7) #trying to bring down loss

    #pos_loss = -np.log(1-sigmoid + 1e-10).sum()
    #neg_loss = -np.sum(np.log(neg_sigmoid + 1e-10))
    pos_loss = -np.log(sigmoid + 1e-10).sum()      #get two loses
    neg_loss = -np.sum(np.log(neg_sigmoid + 1e-10))
    losses.append(pos_loss + neg_loss)

    pos_gradi = (1-sigmoid).reshape(-1, 1) #converts to column vector
    neg_gradi = (neg_sigmoid).reshape(-1, 1)

    update = np.dot(context_v.T, pos_gradi).squeeze() + \
                           np.dot(negative_v.T, neg_gradi).squeeze() #squeeze since shapes didn't match

    grad_norm = np.linalg.norm(update)
    if grad_norm > 10:
        update = (update / grad_norm) * 10 #had exploding gradient problem

    w_vectors[index] -= LR * update  #update target vec, move toward context words away from negs

    c_vectors[contexts_word] -= LR * pos_gradi * targetv
    c_vectors[neg_words] -= LR * neg_gradi * targetv

    if iter%10_000 == 0:
      print(sum(losses)/len(losses)) # This should go down as training proceeds
      losses = [] # resetting log every 10K steps

print('training time:', time.time()-start_time)

In [ ]:
w_vectors_norms = np.linalg.norm(w_vectors, axis=1)
c_vectors_norms = np.linalg.norm(c_vectors, axis=1)

In [ ]:
get_nearest_neighbours('First', vectors=w_vectors, top_k=5, vocab=vocab,
                       norms=w_vectors_norms)

In [ ]:
get_nearest_neighbours('First', vectors=c_vectors, top_k=5, vocab=vocab,
                       norms=c_vectors_norms)

In [ ]:
# evaluation - do not change
start_time = time.time()
hit_at_k = 0.
for a,b,c,d in tqdm(task): # Note down this Inference Time
  d_ = get_nearest_neighbours(
        word_or_vector=w_vectors[revmap[a]]+w_vectors[revmap[b]]-w_vectors[revmap[c]],
        top_k=100,
        vectors=w_vectors,
        norms=w_vectors_norms,
      )
  # Note that we could have used c_vectors as well instead of w_vectors
  if d in d_:
    hit_at_k += 1
hit_at_k/=len(task)
print('inference time:', time.time() - start_time)
print(hit_at_k)

In [ ]:
import sys # Hint: use sys.getsizeof() and convert to MBs

ppmi_memory = sys.getsizeof(ppmi)
print(ppmi_memory)

sgns_memory = sys.getsizeof(w_vectors) + sys.getsizeof(c_vectors)
print(sgns_memory)

# IMPLEMENT HERE

In [ ]:
# do not change
import numpy as np

from abc import ABC, abstractmethod

class NNComp(ABC):
    @abstractmethod
    def forward(self, x):
        pass

    @abstractmethod
    def backward(self, incoming_grad):
        pass


class Linear(NNComp):
    def __init__(self, in_dim, out_dim):
        k = 1 / in_dim
        self.in_dim = in_dim
        self.out_dim = out_dim

        # forward stuff, we need to initialize the weight and the bias
        self.weight = np.random.uniform(-np.sqrt(k), np.sqrt(k), size=(in_dim, out_dim))
        self.bias = np.random.uniform(-np.sqrt(k), np.sqrt(k), size=out_dim)

        # backward stuff
        self.backward_context = None
        self.dweight = np.empty_like(self.weight, dtype=float)
        self.dbias = np.empty_like(self.bias, dtype=float)

    def forward(self, x):
        assert np.shape(x)[-1] == self.in_dim
        self.backward_context = x
        return x @ self.weight + self.bias

    def backward(self, incoming_grad):
        self.dweight = self.backward_context.T @ incoming_grad
        self.dbias = incoming_grad.sum(axis=0)
        return incoming_grad @ self.weight.T

    def update_parameters(self, lr):
        self.weight -= lr * self.dweight
        self.bias -= lr * self.dbias


class LogSoftmax(NNComp):
    def __init__(self):
        self.backward_context = None

    def forward(self, x):
        self.backward_context = x - np.max(x, axis=-1, keepdims=True)
        return self.backward_context - np.log(np.exp(self.backward_context).sum(axis=-1, keepdims=True))

    def backward(self, incoming_grad):
        softmax = np.exp(self.backward_context) / np.exp(self.backward_context).sum(axis=-1, keepdims=True)
        return incoming_grad - softmax * incoming_grad.sum(axis=-1, keepdims=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
class SelfAttention(NNComp):
  def __init__(self, embed_dim):
    # do not change
    self.embed_dim = embed_dim # Specify the embedding dimension

    # Initializer Q, K, V vectors
    self.query_layer = Linear(embed_dim, embed_dim)
    self.key_layer = Linear(embed_dim, embed_dim)
    self.value_layer = Linear(embed_dim, embed_dim)
    self.softmax = LogSoftmax()

  def forward(self, x):
    # Start with setting up the Q, K, V vectors
    # Compute the attention scores using Q, K and normalize them using softmax
    # Combine the attention weights with V
    # Don't forget to store the backward context
    # IMPLEMENT HERE
    Q = self.query_layer.forward(x)
    K = self.key_layer.forward(x)
    V = self.value_layer.forward(x)

    dqk = self.embed_dim
    scores = np.dot(Q, K.T) / np.sqrt(dqk)

    attention_weights = self.softmax.forward(scores)

    output = np.dot(attention_weights, V)

    self.Q, self.K, self.V = Q, K, V
    self.attention_weights = attention_weights
    return output

  def backward(self, incoming_grad):
    # Implement backward for Self-Attention, feel free to use the backward() functions defined for modules above
    # Utilize the gradient formulation provided above
    # IMPLEMENT HERE
    attentions = np.dot(incoming_grad, self.V.T)

    d_scores = self.softmax.backward(attentions)

    dQ = np.dot(d_scores, self.K)
    dK = np.dot(d_scores.T, self.Q)
    dV = np.dot(self.attention_weights.T, incoming_grad)
    dxq = self.query_layer.backward(dQ)
    dxk = self.key_layer.backward(dK)
    dxv = self.value_layer.backward(dV)

    return dxq + dxk + dxv

  def update_parameters(self, lr):
    # Implement update here
    # IMPLEMENT HERE
    self.query_layer.update_parameters(lr)
    self.key_layer.update_parameters(lr)
    self.value_layer.update_parameters(lr)


In [ ]:
# do not change
from datasets import load_dataset

# Load the SuperGLUE WiC dataset
dataset = load_dataset('super_glue', 'wic', trust_remote_code=True)

# Extract the training and testing data
train_data = dataset['train']
test_data = dataset['validation']

# Prepare sentences and labels
sentences_1_train = train_data['sentence1']
sentences_2_train = train_data['sentence2']
labels_train = train_data['label']

sentences_1_test = test_data['sentence1']
sentences_2_test = test_data['sentence2']
labels_test = test_data['label']

In [ ]:
# Example of an instance where the meaning of the word "place" is not the same
sentences_1_train[0], sentences_2_train[0], labels_train[0]

In [ ]:
# Example of an instance where the meaning of the word "summer" is the same
sentences_1_train[4], sentences_2_train[4], labels_train[4]

In [ ]:
# do not change, helper functions to implement padding for w2v embeddings

def normalize_embeddings(embeddings):
  mean = np.mean(embeddings, axis=0)
  std = np.std(embeddings, axis=0)
  return (embeddings - mean) / (std + 1e-8)

def extract_w2v_embeddings(sentences, max_tokens=10):
  embeddings = []
  for sentence in sentences:
    w2v_embeddings_sentence = []
    words = sentence.split()

    for word in words:
      if word in revmap:
        w2v_embeddings_sentence.append(w_vectors[revmap[word]])

    if len(w2v_embeddings_sentence) < max_tokens:
      pad_tokens = max_tokens - len(w2v_embeddings_sentence)
      w2v_embeddings_sentence.extend([np.zeros(D) for i in range(pad_tokens)])
    else:
      w2v_embeddings_sentence = w2v_embeddings_sentence[:max_tokens]

    embeddings.append(np.array(w2v_embeddings_sentence))

  return embeddings

input_embeddings_1 = extract_w2v_embeddings(sentences_1_train)
input_embeddings_2 = extract_w2v_embeddings(sentences_2_train)

In [ ]:
# Example learning rate and number of epochs
lr = 1e-5
epochs = 10

attention_model = SelfAttention(D)

for epoch in range(epochs):
  total_loss = 0
  for i, (emb1, emb2) in enumerate(zip(input_embeddings_1, input_embeddings_2)):
    # IMPLEMENT HERE
    # Forward pass through self-attention model for both sentences

    # Compute similarity score between the two outputs (e.g., cosine similarity)

    # Define target (either 0 or 1)

    # Compute Binary Cross-Entropy Loss

    # Compute gradient of the loss w.r.t. predicted output

    # Backpropagate the gradients through the attention model for both embeddings

    # Update parameters

    output1 = attention_model.forward(emb1)
    output2 = attention_model.forward(emb2)

    dot = np.dot(output1.flatten(), output2.flatten())
    norm1 = np.linalg.norm(output1) # got norms
    norm2 = np.linalg.norm(output2)
    similarity = dot / (norm1 * norm2) #get cos similarity

    predicted = scipy.special.expit(similarity) #sig

    predicted = np.clip(predicted, 1e-5, 1 - 1e-5) #keep

    target = labels_train[i]
    grad_loss = predicted - target
    loss = -((target * np.log((predicted + 1e-10))) + ((1 - target)) * (np.log(1 - predicted + 1e-10)))
    total_loss += loss

    grad1 = (grad_loss) * output1 #backprop gradients
    grad2 = grad_loss * output2

    attention_model.backward(grad1)
    attention_model.backward(grad2)

    attention_model.update_parameters(lr)

  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(input_embeddings_1)}")

In [ ]:
# do not change

# get_sentence_embeddings is a general purpose function that will create sentence embeddings given any type of embeddings
def get_sentence_embeddings(sentences, attention_model=None):
  feature_matrix = np.zeros((len(sentences), D))

  for i, sentence in enumerate(sentences):
    words = sentence.split()

    # We only look for words that occur in our vocabulary (specially since we are using a different dataset to build the vocabulary)
    word_indices = [revmap[word] for word in words if word in revmap]

    # Sentence embeddings are created by aggregating the word embeddings obtained
    if word_indices:
      word_embeddings = w_vectors[word_indices, :]
      if attention_model:
        word_embeddings = attention_model.forward(word_embeddings)
      feature_matrix[i] = np.mean(word_embeddings, axis=0)
    else:
      feature_matrix[i] = np.zeros(D)

  return feature_matrix

In [ ]:
# do not change

# Extract features for both sets of sentences in training and testing
features_1_train_word2vec = get_sentence_embeddings(sentences_1_train)
features_2_train_word2vec = get_sentence_embeddings(sentences_2_train)
features_1_test_word2vec = get_sentence_embeddings(sentences_1_test)
features_2_test_word2vec = get_sentence_embeddings(sentences_2_test)

# Same for contextual embeddings
features_1_train_contextual = get_sentence_embeddings(sentences_1_train, attention_model=attention_model)
features_2_train_contextual = get_sentence_embeddings(sentences_2_train, attention_model=attention_model)
features_1_test_contextual = get_sentence_embeddings(sentences_1_test, attention_model=attention_model)
features_2_test_contextual = get_sentence_embeddings(sentences_2_test, attention_model=attention_model)

In [ ]:
# do not change

import numpy as np

# Combine the features from both sentences for the classifier input
X_train_word2vec = np.concatenate([features_1_train_word2vec, features_2_train_word2vec], axis=1)
X_test_word2vec = np.concatenate([features_1_test_word2vec, features_2_test_word2vec], axis=1)

X_train_contextual = np.concatenate([features_1_train_contextual, features_2_train_contextual], axis=1)
X_test_contextual = np.concatenate([features_1_test_contextual, features_2_test_contextual], axis=1)

In [ ]:
# do not change

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Train classifier on word2vec embeddings
clf_word2vec = LogisticRegression(max_iter=1000)
clf_word2vec.fit(X_train_word2vec, labels_train)
pred_word2vec = clf_word2vec.predict(X_test_word2vec)

# Train classifier on contextual embeddings
clf_contextual = LogisticRegression(max_iter=1000)
clf_contextual.fit(X_train_contextual, labels_train)
pred_contextual = clf_contextual.predict(X_test_contextual)

# Evaluate and compare accuracies
accuracy_word2vec = accuracy_score(labels_test, pred_word2vec)
accuracy_contextual = accuracy_score(labels_test, pred_contextual)

print(f"Word2Vec Embeddings Accuracy: {accuracy_word2vec}")
print(f"Contextual Embeddings Accuracy: {accuracy_contextual}")